In [ ]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyproj import Transformer

In [ ]:
pd.options.display.max_columns = None
plt.rc("figure", figsize=[12, 4])

In [ ]:
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())


## Analyse

👉 RÉSUMÉ EN FIN DE PAGE

Documentation du fichier : https://www.data.gouv.fr/fr/datasets/r/d06a0924-9931-4a60-83b6-93abdb6acfd6

### Importation des données bruts

Le fichier utilisé n'est pas le fichier brut produit par FINESS (https://www.data.gouv.fr/fr/datasets/finess-extraction-du-fichier-des-etablissements/), mais une version pré-nettoyée via ce [script](https://github.com/taniki/notebooks/blob/master/finess/clean.ipynb).

In [ ]:
raw_df = pd.read_csv(
    os.environ.get("FINESS_FILE_URL"),
    sep=",",
    index_col=0,  # la 1ère ligne du fichier est le numéro de ligne
    on_bad_lines="warn",
    dtype=str,
)


### Aperçu

In [ ]:
raw_df.sample(2)

In [ ]:
raw_df.shape[0]

In [ ]:
# Le fichier contient des catégories d'établissements.
# Seul un sous-ensemble de ces catégories nous intéresse.
# cf https://www.notion.so/dora-beta/Analyse-des-donn-es-FINESS-75b23111f35a4057a97ff4e2bb1fa78f

raw_df = raw_df[
    raw_df["categagretab"].isin(['4301', '4302', '4303', '4501', '4601', '4602', '4607'])
    | (raw_df["categagretab"] == "2202") & (raw_df["categetab"] == "228")
    | (raw_df["categagretab"] == "2206") & (raw_df["categetab"] == "636")
]

raw_df.shape[0]

In [ ]:
# Labels sont issus de la documentation FINESS
# cf https://www.notion.so/dora-beta/Analyse-des-donn-es-FINESS-75b23111f35a4057a97ff4e2bb1fa78f

categories = {
    '4301': "Etab. et Services d'Hébergement pour Adultes Handicapés",
    '4302': "Services de Travail Protégé pour Adultes Handicapés",
    '4303': "Services de Réinsertion Prof pour Adultes Handicapés",
    '4501': "Etablissements de l'Aide Sociale à l'Enfance",
    '4601': "Etablissements pour Adultes et Familles en Difficulté",
    '4602': "Autres Etablissements Sociaux d'Hébergement et d'Accueil",
    '4607': "Logements en Structure Collective",
}

sub_categories = {
    '228': "Centre Planification ou Education Familiale Ctre.Planif.Educ.Fam",
    '636': "Centre de soins et de prévention Centre soins prév.",
}

### nettoyage

In [ ]:
raw_df = raw_df.replace(["", np.nan], None)

### Taux de remplissage des champs de structure

In [ ]:
def compute_field_occupancy_rates(df):
    return ((1 - df.isnull().sum() / df.shape[0]) * 100).sort_values(ascending=False)

compute_field_occupancy_rates(raw_df).to_frame()

### Identifiant local ?

2 champs potentiels : `nofinesset` et `nofinessej`

In [ ]:
raw_df.nofinessej.nunique()

In [ ]:
raw_df.nofinesset.nunique()

conclusion : `nofinessej` dupliqué et `nofinesset` unique -> `nofinesset` identifiant local

### Code insee ?

In [ ]:
raw_df.assign(foo=raw_df.ligneacheminement.str.extract(r"\d{5} (.*?)(?= CEDEX|$)"))[["foo", "ligneacheminement"]][raw_df.ligneacheminement.str.contains("CEDEX")]

In [ ]:
raw_df[["departement", "commune", "ligneacheminement"]].sample(5)
raw_df[raw_df.nofinesset == "2A0001269"]

In [ ]:
raw_df[raw_df.nofinesset == "970407573"]

In [ ]:
raw_df[raw_df.nofinesset == "970407573"].iloc[0].to_json(force_ascii=False)

conclusion: departement + commune = code_insee

### Date de màj ?

2 champs potentiels : `maj` et `datemaj`

In [ ]:
raw_df.datemaj.value_counts()

In [ ]:
plt.rc("figure", figsize=[12, 4])
sns.set(style="darkgrid")
sns.histplot(data=raw_df.maj.apply(pd.to_datetime, errors="coerce"), bins=20)

conclusion:
* `datemaj`: la même date `2022-07-04` --> pas super fiable...
* `maj`: plus intéressant, mais quelle diff entre les 2 champs ?

### SIRET ?

In [ ]:
raw_df.siret.value_counts().head(10).to_frame()

In [ ]:
# "31723624800017" -> Emmaüs
# "77568030900611" -> Coallia
# "78805803000016" -> Adoma
# ...

In [ ]:
raw_df.siret.drop_duplicates(keep=False).count()

* Pas mal de réseaux et d'antennes
* 9363 structures uniques

### Nettoyage

In [ ]:
df = raw_df.copy()

# Traitement des dates
df[["dateouv", "dateautor", "maj", "datemaj"]] = df[
    ["dateouv", "dateautor", "maj", "datemaj"]
].apply(pd.to_datetime, errors="coerce")

# Reconstruction des adresses
df["adresse"] = (
    df.numvoie.fillna(0.0).astype(int).astype(str).replace(0, None)
    + " "
    + df.typvoie
    + " "
    + df.voie
)

# Dissocier les codes postaux des noms de ville
df[["cp", "commune"]] = df.ligneacheminement.str.split(" ", 1, expand=True)


In [ ]:
# Conversion des coordonnées du système géodésique local vers WGS84
EPSG = {
    # Default value : 2154,
    'GUADELOUPE': 2970,
    'MARTINIQUE': 2970,
    'GUYANE': 2972,
    'LA REUNION': 2975,
    'MAYOTTE': 4471,
    'SAINT PIERRE ET MIQUELON': 4467
}

def coordinates_to_wgs84(df, from_epsg):
    transformer = Transformer.from_crs("epsg:" + str(from_epsg), "epsg:4326", always_xy=True)
    latitude, longitude = transformer.transform(df['coordxet'], df['coordyet'])
    return latitude, longitude

# Transformer les coordonnées de la métropole
wgs84 = coordinates_to_wgs84(df, 2154)
mask = ~df.libdepartement.isin(EPSG.values())
df.loc[mask, 'longitude'] = wgs84[0][mask]
df.loc[mask, 'latitude'] = wgs84[1][mask]

# Transformer les coordonnées des Territoires d'Outre-Mer
for location, code in EPSG.items():
    wgs84 = coordinates_to_wgs84(df, code)
    mask = df.libdepartement == location
    df.loc[mask, 'longitude'] = wgs84[0][mask]
    df.loc[mask, 'latitude'] = wgs84[1][mask]

# Analyse des typologies

In [ ]:
from data_inclusion.schema import models

In [ ]:
# Recherche des typologies remarquables
for typ in models.Typologie:
    if df['rs'].apply(lambda s: typ.value.lower() in s.lower().split()).any():
        print('')
        print(typ.value)
        print(df[df['rs'].apply(lambda s: typ.value.lower() in s.lower().split())].sample(1).rs)

In [ ]:
categories_flags_places_df = (
    df.rs.str.lower()
    .apply(
        lambda s: {
            "ase": "ase" in s.split(),
            "association": "association" in s.split() or "asso" in s.split(),
            "cada": "cada" in s.split(),
            "cava": "cava" in s.split(),
            "ccas": "ccas" in s.split(),
            "chrs": "chrs" in s.split(),
            "chu": "chu" in s.split(),
            "cias": "cias" in s.split(),
            "cidff": "cidff" in s.split(),
            "csapa": "csapa" in s.split(),
            "ea": "ea" in s.split(),
            "esat": "esat" in s.split(),
            "huda": "huda" in s.split(),
            "mde": "mde" in s.split(),
            "mdef": "mdef" in s.split(),
            "mjc": "mjc" in s.split(),
            "msa": "msa" in s.split(),
            "pension": "pension" in s.split(),
            "prevention": "prevention" in s.split(),
            "cph": "cph" in s.split(),
            "udaf": "udaf" in s.split(),
        },
    )
    .apply(pd.Series)
    .assign(
        na=lambda df: df.apply(
            lambda row: ~row.any(), axis="columns", result_type="expand"
        )
    )
)

categories_flags_places_df.sum().sort_values(ascending=False).plot(kind="bar", grid=True, rot=35, figsize=(20, 8))

In [ ]:
plt.rc("figure", figsize=[12, 8])
sns.countplot(
    data=df.fillna('Inconnu'),
    y='libcategetab',
    order = df['libcategetab'].fillna('Inconnu').value_counts().index
)

### Etude des répartitions géographiques

In [ ]:
plt.rc("figure", figsize=[12, 16])
sns.countplot(
    data=df.fillna('Inconnu'),
    y='libdepartement',
    order = df['libdepartement'].fillna('Inconnu').value_counts().index
)

## Résumé

<a id='summary'></a>

<br>✅ Dataset de structures
<br>✅ 100% de dates de création et de mises à jour
<br>✅ 100% d'identifiants uniques ('nofinessej')
<br>✅ 100% de noms
<br>✅ 100% de communes
<br>✅ Les champs peuvent être majoritairement remplis proprement
<br>
<br>⚠️ 84% de SIRET
<br>⚠️ Voies et noms de rue spécifiques à considérer
<br>⚠️ Nombreuses typologies présentes, demande une analyse plus fine
<br>
<br>❌ Adresses parfois partielles
<br>❌ Aucun rna